In [6]:
# =============================================================================
# K Means: segmenting neighborhoods in Toronto
# =============================================================================
from bs4 import BeautifulSoup
import requests
import pandas as pd


# assign the link of the website to scrape the data
website_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
resp = requests.get(website_url).text

#Beautiful Soup to Parse the url page
soup = BeautifulSoup(resp, 'xml')

table = soup.find('table')

column_names=['Postalcode','Borough','Neighbourhood']
df = pd.DataFrame(columns=column_names)

# extracting information from the table
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data    

# remove rows where Borough is 'Not assigned'
df=df[df['Borough']!='Not assigned']

# assign Neighbourhood=Borough where Neighbourhood is 'Not assigned'
df[df['Neighbourhood']=='Not assigned']=df['Borough']

df.head()

# group multiple Neighbourhood under one Postcode
temp_df=df.groupby('Postalcode')['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighbourhood':'Neighbourhood_joined'},inplace=True)

# join the newly constructed joined data frame
df_merge = pd.merge(df, temp_df, on='Postalcode')

# drop the Neighbourhood column
df_merge.drop(['Neighbourhood'],axis=1,inplace=True)

# drop duplicates from the data frame
df_merge.drop_duplicates(inplace=True)

# rename Neighbourhood_joined back to Neighbourhood
df_merge.rename(columns={'Neighbourhood_joined':'Neighbourhood'},inplace=True)

df_merge.head()

,Postalcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
4,M6A,North York,"Lawrence Heights, Lawrence Manor"
6,Queen's Park,Queen's Park,Queen's Park


In [7]:
df_merge.shape

(103, 3)